## Animating Atmospheric Rivers 
##### Lander Greenway, Cole Pietsch

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cmocean
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point
import ftplib
import os
import imageio.v3 as iio
import datetime
from glob import glob

#### *Define parameters for animation*

In [2]:
# Define start and end dates
start = '2023-08-22'
end = '2023-08-24'

#define other parameters, 'tp' for precip and '
var = 'tp'

In [3]:
# load data
data = xr.open_dataset('/Users/colepietsch/Documents/EV333_AtmosphericDynamics/Rivers/atmorivers/ERA5_SouthAmerica_2023_hourly_tp.nc')

**-----------------------------------------------------------------------------------------------------------------------------------------**

In [4]:
#output directory
output_dir = str(start + '_' + end)
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
else:
    print("Directory '{}' already exists. Skipping creation.".format(output_dir))

Directory '2023-08-22_2023-08-24' already exists. Skipping creation.


In [5]:
#subset data and extract max for scaling color bar
data = data.sel(time = slice(start, end))
max_value = data[var].max().values.item()
#max_value = np.atleast_1d(max_value).item()
tick_value = max_value / 40

#extracting variable and applying unit conversions if needed
if var == 'tp' :
    data = data[var] * 1000
    title = 'Total Precipitation'
    lev = np.arange(0, (1000 * max_value) * 0.75, tick_value * 1000) 
    cmap = cmocean.cm.rain  
    cbar_lab = "Total Precip. in mm/hr"

if var == 'tcvw' : 
    data = data[var]
    title = 'Total Water Column Vapor Transport'
    lev = np.arange(0, max_value, tick_value)
    cmap = cmocean.cm.rain
    cbar_lab = "Total Precip. in mm/hr"

#defining number of frames given by subset data
num = len(data['time'])

In [6]:
#choosing projection
proj = ccrs.PlateCarree(central_longitude=-77)

#create loop
for i in range(0, 1) :

    # define figure and axes, figure size, and resolution
    fig = plt.figure(figsize=(8, 4.5), dpi=300)
    ax = plt.axes(projection = proj)

    # filled contour map of metric in question
    data[i].plot.contourf(
        x = 'longitude',
        y = 'latitude',
        ax=ax,
        transform=ccrs.PlateCarree(),
        levels=lev,
        extend='max',
        colors=cmap,
        add_colorbar=True,
        cbar_kwargs = {"label":cbar_lab})

    # add coastlines
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.coastlines(resolution='110m')  #Currently can be one of “110m”, “50m”, and “10m”.

    # add grid lines
    gl = ax.gridlines(crs=ccrs.PlateCarree(),
                      draw_labels=True,
                      linewidth=1,
                      color='gray',
                      alpha=0.5,
                      linestyle='--')
    gl.right_labels = None
    gl.top_labels = None
    

    # Extract year, month, day, and hour from time series, splits are used on format of 'YYYY-MM-DDT00:00:00.000000000'
    times = data['time'][i].values.astype('datetime64[s]')
    yr = str(np.datetime64(times, 'Y').astype(str))
    mon = str(np.datetime64(times, 'M').astype(str).split('-')[1])
    day = str(np.datetime64(times, 'D').astype(str).split('-')[2])
    hr = str(np.datetime64(times, 'h').astype(str).split('T')[1].split(':')[0])
   
    # Construct the filename using the extracted information
    name = output_dir + f'/{var}_{yr}-{mon}-{day}-{hr}.png'

    # add title
    ax.set_title(title + ' on ' + mon + '/' + day + ', ' + yr + ' at ' + hr + ':00')
  
    # Save the figure
    fig.savefig(name, facecolor='white', transparent=False, bbox_inches='tight')


    
    plt.close(fig)


In [7]:
#getting frames 
files = sorted(glob(output_dir + '/' + var + '*.png'))

#creating empty array
images = [ ]

#setting loop to append files to images
for filename in files :
    images.append(iio.imread(filename))
    
#writing gif
iio.imwrite(var + '_' + start + '_' + end +'.gif', images, duration = 250, loop = 0)